# Project Work n.2

---


# Corso Diploma di Esperto di Data Science presso Scuola Nazionale dell'Amministrazione (SNA)
# GdL: Patrizia Mazzucchi, Maurizio D'Amato


---


##TRACCIA
Project work sulla acquisizione di una base di dati (da pulire, ordinare,inquadrare) usabile per la produzione di statistiche
Visitare il seguente sito: https://dati.anticorruzione.it/opendata/dataset/ocds-appalti-ordinari-2022
Scaricare a scelta uno dei dataset linkati (la relativa documentazione di formato è disponibile all’URL https://dati.anticorruzione.it/opendata/ocds_it )
Produrre uno script in Python o R che :
- evidenzi la presenza di eventuali valori scorretti nei dati (es.: data fine lavori precedente data inizio, appalti senza descrizione o titolo, valori fuori scala, ecc.);
- produca statistiche di tipo descrittivo sui dati (es.: distribuzione degli importi, distribuzione dei bandi e degli importi per fornitore, ecc.);
- produca grafici di sintesi sui fenomeni ritenuti di maggiore interesse.
Confrontare i risultati ottenuti con i medesimi relativi al mese precedente quello di riferimento inizialmente scelto.
Per un supporto nell’accesso, uso e/o il trattamento delle basi di dati scrivere al dott. Alessio Cimarelli: alessio@dataninja.it.


Importo le librerie che mi servono e imposto le configurazioni globali:
per leggere e scrivere csv e json
- [pandas](https://pandas.pydata.org/) per la manipolazione e l'analisi dei dati
- [matplotlib](https://matplotlib.org/) per la configurazione dei grafici

In [ ]:
# librerie standard 
import csv
#import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab as pl
#plt.style.use("ggplot")

Scarico e importo su google drive dal sito [ANAC](https://dati.anticorruzione.it/opendata/ocds_it) il dataset csv [CIG_csv_2022_05](https://dati.anticorruzione.it/opendata/dataset/cig-2022/resource/0df8a602-8cfd-4598-be61-0f854bb5aab3) che contiene tutte le informazioni relative agli appalti ordinari pubblicati comunicati nel Maggio 2022, per cui è stato emesso un codice CIG. 


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/file/d/1PSsBdc5McG3pPIlCqVNMXk1CzSMTnMNA/view?usp=sharing'

# to get the id part of the file
id = link.split("/")[-2]
#scarico il file da google drive
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('cig_csv_2022_05.csv')

#leggo il file e lo metto in un dataframe
df = pd.read_csv('cig_csv_2022_05.csv', sep=';')

#**DATA EXPLORATION, PRE-PROCESSING (DATA CLEANING , DATA REDUCTION): **
lo scopo è di individuare dati mancanti, outliers e pre-processare / trasformare alcune feature. Nonchè mettere da parte dati che non sono rilevanti per lo scopo.
Raccolgo un po' di informazioni sul dataset, caricato sul dataframe lo faccio con alcune funzioni utili come *head* *info* e *describe*

In [ ]:
# utlizziamo la funzione head per iniziare la fase di analisi e pulitura del dataset
df.head()

In [ ]:
# utlizziamo la funzione info per iniziare la fase di analisi e pulitura del dataset
df.info()


In [ ]:
# verifichiamo che lo stato sia per tutti attivo
df.stato.unique()


dalla funzione *info* si può dedurre che alcune colonne hanno parecchi valori nulli (*cig_accordo_quadro, luogo_istat, provincia*), si stabilisce quindi di eliminarli dal nostro dataframe da analizzare in quanto non utili all'obiettivo.
anno_pubblicazione mese_pubblicazione codice_ausa cod_cpv descrizione_cpv flag_prevalente non sono rilevanti, li cancelliamo

In [ ]:
# forziamo il tipo dato degli importi ad intero, erano float 
df.importo_lotto = df.importo_lotto.astype(int)
df.importo_complessivo_gara = df.importo_complessivo_gara.astype(int)
df.info()

In [ ]:
# Cancelliamo le colonne che non riteniamo utili, le province non sono tutte valorizzate, ci basiamo sulle regioni
df.drop(['cig_accordo_quadro', 'luogo_istat', 'provincia', 'anno_pubblicazione','mese_pubblicazione','codice_ausa','cod_cpv','descrizione_cpv','flag_prevalente'], axis = 1, inplace = True)
df.info()

ora abbiamo solo 21 colonne/feature delle 30 iniziali. Volendo soffermarci sulla suddivisione regionale analizzo la colonne *sezione_regionale*

In [ ]:
#verifichiamo i differenti valori nella sezione regionale
df.sezione_regionale.unique()

In [ ]:
# analizziamo i valori nulli
regioni_mancanti = pd.isnull(df['sezione_regionale'])

df[regioni_mancanti]


analizziamo nel dettaglio la feature "sezione regionale" con l'obiettivo di visualizzare i contratti suddivisi per regione. Dalla funzione info vediamo che mancano 12  valori (31929-31917=12). Visualizziamoli e cancelliamo le occorrenze che non sono valorizzate.

In [ ]:
 # cancelliamo solo le righe che hanno valori nulli nella colonna "sezione_regionale" con il metodo dropna
df.dropna(subset=['sezione_regionale'], inplace=True)
df.info()

abbiamo ora 21 colonne e 31.917 righe delle 31.929 iniziali. verifichiamo che l'operazione sia andata a buon fine. 

In [ ]:
# contolliamo che non ci siano record con valori nulli nel campo sezione_regionale
regioni_mancanti = pd.isnull(df['sezione_regionale'])
df[regioni_mancanti]

In [ ]:
df['sezione_regionale'].describe()

Dalla funzione describe() emerge che la sezione regionale che ha più contratti nel mese di maggio 2022 è quella piemontese con 5.646 contratti/lotti attivi.

In [ ]:
# visualizziamo le occorrenze con sezione_regionale = "NON CLASSIFICATO", sono 813
regioni_NC=df['sezione_regionale']=='NON CLASSIFICATO'
df[regioni_NC]

In [ ]:
# elimino le occorrenze con sezione_regionale = "NON CLASSIFICATO"
df = df.drop(df[df.sezione_regionale == "NON CLASSIFICATO"].index)

In [ ]:
#verifichiamo che non esistano piu' i valori nulli e "NON CLASSIFICATO"
df['sezione_regionale'].unique()

In [ ]:
#le occorrenze ora sono 31.104 avendo eliminato anche i valori "NON CLASSIFICATO" che erano 813
df['sezione_regionale'].describe()

In [ ]:
# modifichiamo i valori della colonna sezione_regionale, rendendoli più leggibili
df['sezione_regionale'] = df['sezione_regionale'].map(
                            {'SEZIONE REGIONALE LOMBARDIA':'LOMBARDIA',
                             'SEZIONE REGIONALE FRIULI VENEZIA GIULIA':'FRIULI VENEZIA GIULIA',
                             'SEZIONE REGIONALE CAMPANIA':'CAMPANIA',
                             'SEZIONE REGIONALE LIGURIA':'LIGURIA',
                             'SEZIONE REGIONALE VENETO':'VENETO',
                             'SEZIONE REGIONALE PIEMONTE':'PIEMONTE',
                             'SEZIONE REGIONALE CENTRALE':'SEZIONE CENTRALE',
                             'SEZIONE REGIONALE UMBRIA':'UMBRIA',
                             'SEZIONE REGIONALE SICILIA':'SICILIA',
                             'SEZIONE REGIONALE SARDEGNA':'SARDEGNA',
                             'SEZIONE REGIONALE PUGLIA':'PUGLIA',
                             'SEZIONE REGIONALE TOSCANA':'TOSCANA',
                             'SEZIONE REGIONALE MARCHE':'MARCHE', 
                             'SEZIONE REGIONALE BASILICATA':'BASILICATA', 
                             'SEZIONE REGIONALE EMILIA ROMAGNA':'EMILIA ROMAGNA', 
                             'SEZIONE REGIONALE CALABRIA':'CALABRIA', 
                             'SEZIONE REGIONALE LAZIO':'LAZIO', 
                             'SEZIONE REGIONALE MOLISE':'MOLISE', 
                             'SEZIONE REGIONALE ABRUZZO':'ABRUZZO',
                             "SEZIONE REGIONALE VALLE D'AOSTA":'VALLE D AOSTA',
                             'SEZIONE REGIONALE PROVINCIA AUTONOMA DI TRENTO':'PROVINCIA AUTONOMA DI TRENTO',
                             'SEZIONE REGIONALE  PROVINCIA AUTONOMA DI BOLZANO':'PROVINCIA AUTONOMA DI BOLZANO'},
                             na_action=None
                             )

In [ ]:
#verifichiamo che la sostituzione sia stata effettuata
df['sezione_regionale'].unique()

# **DATA VISUALIZATION: dopo aver pulito inizialmente i dati proviamo a fare qualche grafico che dia una idea della distribuzione del contratti su base regionale e sulla base di altri criteri**

In merito alle 11 regole della statistica e delle visualizzazione dati, di seguito alcune considerazioni sulle quelle applicabili:
1. sarebbe utile confrontare i valori regionali in proporzione al rispettivo territorio e alla popolazione.
2. il dataset cig_csv_xxx.csv contiene dati riferiti ad un mese e solo delle attività contrattuali che hanno richiesto l'emissione del codice CIG. Non sono compresi in queste i contratti sottosoglia che hanno richiesto l'emissione del codice SMART CIG.
3. la fonte è da considerare attendibile in quanto Autorità Nazionale Anti-Corruzione.
4. Il campione è sufficientemente rappresentativo, anche se limitato temporalmente alla produzione di un mese.

9. più che effettuare classifiche confrontiamo valori tra le varie voci, valutando la percentuale sul totale.
10. la fonte è ANAC.


## IMPORTI

In [ ]:

# da questa funzione si deduce che l'importo massimo di una gara è di 900M e ci sono gare con importo 0, la media è di circa 3.8M.
#i lotti invece hanno una massima numerosità di n.149 e minimo 1, sono in media circa 6. L'importo del lotto massinmo è 534M e possono essere uguale a 0.
df.describe()


In [ ]:
# visualizziamo gli importi gara = 0
importi_0 = df.importo_complessivo_gara == 0
df[importi_0]

## **visualizziamo gli importi gara = 0, sono n.62, associati a contratti di aziende sanitarie piemontesi e sarde. Potrebbe non essere un errore ed esserci delle ragioni tecnico-amministrative o contabili che giustifichino questo dato a 0. **

In [ ]:
#verifichiamo che gli importi siano sopra soglia,
importi_sottosoglia = df.importo_complessivo_gara < 40000
df[importi_sottosoglia]

## nonostante il codice SMART CIG sia dedicato agli importi sottosoglia, ci sono nel file  CIG n. 7.870 importi di gara sottosoglia

In [ ]:
#totale importo gare
df.importo_complessivo_gara.sum()

##**l'importo complessivo delle gare nel maggio 2022 supera i 118 Miliardi €**

#OGGETTO PRINCIPALE CONTRATTO

In [ ]:
# verifichiamo il numero dei contratti per oggetto principale
df.oggetto_principale_contratto.value_counts()

In [ ]:
#grafico oggetto principale contratto
plt.figure(figsize=(20,5))
plt.subplot(121)
raggr_obj2 = df.groupby('oggetto_principale_contratto')['importo_complessivo_gara']
raggr_obj2.count().plot.barh(stacked=True, title="OGGETTO PRINCIPALE CONTRATTO PER NUMERO CONTRATTI)", xlabel="", grid=True)

l'oggetto principale del contratto nella sua aggregazione per numeri di contratti ha una prevalenza della tipologia di **FORNITURE**

In [ ]:
# verifichiamo gli importi dei contratti per oggetto principale
raggr_obj= df.groupby('oggetto_principale_contratto')['importo_complessivo_gara']
raggr_obj.sum().sort_values(ascending=False)


In [ ]:
#grafico oggetto principale contratto
plt.figure(figsize=(20,5))
plt.subplot(122)
raggr_obj3 = df.groupby('oggetto_principale_contratto')['importo_complessivo_gara']
raggr_obj3.sum().plot.barh(stacked=True, title="OGGETTO PRINCIPALE CONTRATTO PER IMPORTO CONTRATTI - unità 10 Miliardi €", xlabel="", grid=True)

l'importo delle gare relativo alle forniture (77 Miliardi €) è nettamente superiore alla somma delle altre due tipologie (servizi e lavori). E' evidente che le commesse relative alle forniture hanno importi mediamente più alti.

##SCELTA CONTRAENTE

In [ ]:
# verifichiamo il numero di contratti per tipo scelta contraente
df.tipo_scelta_contraente.value_counts()

In [ ]:
#grafico scelta contraente per numero contratti
plt.figure(figsize=(20,15))
plt.subplot(121)
raggr_obj2 = df.groupby('tipo_scelta_contraente')['importo_complessivo_gara']
raggr_obj2.count().sort_values(ascending=True).plot.barh(stacked=True, title="SCELTA CONTRAENTE PER NUMERO DI CONTRATTI", xlabel="", grid=True)

nel periodo analizzato le procedure prevalenti per la scelta del contraente sono:
> **AFFIDAMENTO DIRETTO**  
> **AFFIDAMENTO DIRETTO** IN ADESIONE AD ACCORDO QUADROCONVENZIONE                                   
> PROCEDURA NEGOZIATA PER AFFIDAMENTI SOTTOSOGLIA        
PROCEDURA APERTA               

In [ ]:
# verifichiamo gli importi dei contratti per tipo scelta contraente
raggr_obj= df.groupby('tipo_scelta_contraente')['importo_complessivo_gara']
raggr_obj.sum().sort_values(ascending=False)

In [ ]:
#grafico scelta contraente per importo contratti
plt.figure(figsize=(20,15))
plt.subplot(122)
raggr_obj3 = df.groupby('tipo_scelta_contraente')['importo_complessivo_gara']
raggr_obj3.sum().sort_values(ascending=True).plot.barh(stacked=True, title="SCELTA CONTRAENTE PER IMPORTO CONTRATTI - unità 10 Miliardi €", xlabel="", grid=True)

interessante notare che guardando gli importi la "**PROCEDURA APERTA**", risultata quarta come numerosità, ha un valore totale molto alto (82.5 Miliardi €), 2/3 del totale.

##MODALITA' REALIZZAZIONE

In [ ]:
# verifichiamo il numero di contratti per modalità realizzazione
df.modalita_realizzazione.value_counts()

In [ ]:
#grafico modalità realizzazione per numero contratti
plt.figure(figsize=(20,15))
plt.subplot(121)
raggr_obj2 = df.groupby('modalita_realizzazione')['importo_complessivo_gara']
raggr_obj2.count().sort_values(ascending=True).plot.barh(stacked=True, title="MODALITA REALIZZAZIONE PER NUMERO DI CONTRATTI", xlabel="", grid=True)

la maggior parte, circa 70/75%, dei contratti sono realizzati con due modalità "**CONTRATTO D'APPALTO**" e "**CONTRATTO D'APPALTO DISCENDENTE DA ACCORDO QUADRO/CONVENZIONE SENZA SUCCESSIVO CONFRONTO COMPETITIVO**" 

In [ ]:
# verifichiamo gli importi dei contratti per modalità realizzazione
raggr_obj= df.groupby('modalita_realizzazione')['importo_complessivo_gara']
raggr_obj.sum().sort_values(ascending=False)

In [ ]:
#grafico modalità realizzazione per importo contratti
plt.figure(figsize=(20,15))
plt.subplot(122)
raggr_obj3 = df.groupby('modalita_realizzazione')['importo_complessivo_gara']
raggr_obj3.sum().sort_values(ascending=True).plot.barh(stacked=True, title="MODALITA REALIZZAZIONE PER IMPORTO CONTRATTI - unità 10 Miliardi €", xlabel="", grid=True)

anche dagli importi delle modalità di realizzazione della gara si può dedurre che l'**ACCORDO QUADRO** pur non avendo un numerosità alta, rappresenta con quasi 55 Miliardi € quasi il 50% dell'importo totale. Anche la **CONVENZIONE** consta di più di 21 Miliardi a fronte di circa 800 contratti. 

#SUDDIVISIONE PER REGIONI

In [ ]:
# sommiamo il numero di contratti per regione
df.sezione_regionale.value_counts()


In [ ]:
#grafico suddiviso per regioni per numero contratti
plt.figure(figsize=(20,15))
plt.subplot(121)
raggr_obj2 = df.groupby('sezione_regionale')['importo_complessivo_gara']
raggr_obj2.count().sort_values(ascending=True).plot.barh(stacked=True, title="SUDDIVISIONE REGIONALE PER NUMERO DI CONTRATTI", xlabel="", grid=True)

In [ ]:
# confrontiamo in tabella le regioni per importo in Euro complessivo di gara. 
raggr_reg= df.groupby('sezione_regionale')['importo_complessivo_gara']
raggr_reg.sum().sort_values(ascending=False)

In [ ]:
#grafico suddiviso per regioni per importo contratti
plt.figure(figsize=(20,15))
plt.subplot(122)
raggr_obj3 = df.groupby('sezione_regionale')['importo_complessivo_gara']
raggr_obj3.sum().sort_values(ascending=True).plot.barh(stacked=True, title="SUDDIVISIONE REGIONALE PER IMPORTO CONTRATTI - unità Miliardo di €", xlabel="", grid=True)

Le **regioni Piemonte e Lombardia** hanno numeri di contratti nettamente superiore, in ragione anche delle dimensioni. Molto elevato è il numero di contratti nell'ambito delle PA centrali, n.5.190.
Se valutiamo l'importo complessivo, il procurement centrale ha il primato seguito a ruota dalla **Regione Toscana** con quasi 23 miliardi di €. Le altri regioni che superano i 10 miliardi di € sono l**'Abruzzo, la Sicilia, il Lazio e la Campania**. 
